<a href="https://colab.research.google.com/github/alihammadbaig/pdf_to_excel/blob/master/pdf_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
pip install tika

  Stored in directory: /root/.cache/pip/wheels/b4/db/8a/3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c1036848
Successfully built tika


In [0]:
# from shutil import copyfile

# copyfile("/tika-server-1.18.jar", "/tmp//tika-server.jar")

In [0]:
# os.environ["TIKA_PATH"] = "/tmp/tika-server-1.18.jar"
# os.environ["TIKA_SERVER_JAR"] = "/tmp/tika-server-1.18.jar"
# print(os.environ["TIKA_SERVER_JAR"])


In [0]:
import os
os.environ['TIKA_SERVER_JAR'] = 'https://repo1.maven.org/maven2/org/apache/tika/tika-server/1.19/tika-server-1.19.jar'
import tika
from tika import parser

In [0]:
# Text parsing library
# import tika
# from tika import parser

# Deal with regular expression(text processing)
import re

# Handle the excel spreadsheet
import openpyxl

import os

tika.initVM()

# Read the ;PDF file
# parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/sample_pdf_for_excel.pdf')
parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_100OF107_IPX Reviewed_AC-OCR.pdf')

# parsed = parser.from_file('/content/gdrive/My Drive/Inpex files /L290-AQ-MDR-13850_0_SECTN_89OF107_IPX Reviewed_AC-OCR.pdf')
content = parsed["content"].strip()

# Read the spreadsheet for later use
# wb = openpyxl.load_workbook("/content/gdrive/My Drive/Colab Notebooks/data/excel_instructions.xlsx")
wb = openpyxl.load_workbook("/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_100OF107_IPX Reviewed_AC-OCR.xlsx")
sheet = wb['Template']

# Starting writing the sheet beginning line#2
sheet_row_num = 2

# PDF columns
col_names = ["top_coat_date", "blast_primer_date"
             ,"dft1_ave", "dft1_5", "dft1_4", "dft1_3", "dft1_2", "dft1_1"
             ,"dft2_ave", "dft2_5", "dft2_4", "dft2_3", "dft2_2", "dft2_1"
             , "sys_tem"]

# Spreadsheet cells
cells = ["AL", "S", "AC", "AB", "AA", "Z", "Y", "X", "K", "J", "I", "H", "G", "F", "D"]

cols = {}

In [0]:

# page_num_printed = 5479
page_num_printed = 5962
page_num = 1

total_pages = int(parsed["metadata"]["xmpTPg:NPages"].strip())
# total_pages = 5
# print(total_pages)

def get_col():
  global r
  r = r.strip()
  val = r[r.rfind(" ") : len(r)]
  r = r[0 : r.rfind(" ")]
  return val

def hasNumbers(inputString):
  return bool(re.search(r'\d', inputString))


In [30]:
# page_num_printed = 5479
page_num_printed = 10795
page_num = 1

total_pages = int(parsed["metadata"]["xmpTPg:NPages"].strip())
# total_pages = 5
# print(total_pages)

def get_col():
  global r
  r = r.strip()
  val = r[r.rfind(" ") : len(r)]
  r = r[0 : r.rfind(" ")]
  return val

def hasNumbers(inputString):
  return bool(re.search(r'\d', inputString))

import traceback

'''
## Pages ##
'''
# print(page_num)
# print(total_pages)

while True:
  skip = 11
# for page_num in range(1, total_pages):
  print("Page#%i"%page_num)
  print("Page num printed#%i"%page_num_printed)
  
  start_of_page = "Page " + str(page_num_printed) +" of 13699"
  end_of_page = "Page " + str(page_num_printed + 1) +" of 13699"
  page_content = content[content.find(start_of_page): content.find(end_of_page)]
  
#   print("----- Start: Page Content of page:{} -----".format(page_num_printed))
#   print(page_content)
#   print("----- End: Page Content of page:{} -----".format(page_num_printed))
  
  ## Extract the table
  try:
    p = re.compile("(A*VE\\s+DATE)", re.IGNORECASE)
    for m in p.finditer(page_content):
      start_of_table = m.start()+8

    end_of_table = re.search("(FINAL\\s+)*INSPECTION", page_content, flags = re.I)
#     print("1-End of table:{}".format(end_of_table))
    if end_of_table is None:
      end_of_table = re.search("(VISUAL\\s+)*INSPECTION", page_content, flags = re.I)
      skip = 29
#       print("2-End of table:{}".format(end_of_table))
    
    if end_of_table is None: 
      end_of_table = re.search("\\(*REMARK", page_content, flags = re.I)
      skip = 50
#       print("3-End of table: {}".format(end_of_table))
    
    end_of_table = end_of_table.start() - skip
#     print("End of table:{0} and skip:{1}".format(end_of_table,skip))
    
    table = page_content[start_of_table:end_of_table]

    # Data cleansing
    table = table.replace("·","-")\
      .replace("•","")\
      .replace(" ' "," ")\
      .replace("l4", "14")\
      .replace(",", "")\
      .replace(" . ", " ")\
      .replace(" - ", " ")
    
    table = re.sub("OS-([A-Z]{3})", r"05-\1", table,  flags=re.I)
    table = re.sub("(\d{1,2})\.*-\.*([A-Z]{3})\.*-\.*(\d{2,4})", r"\1-\2-\3", table,  flags = re.I)
    table = re.sub("\s+(\d{1,2})\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" \1-\2-\3 ", table, flags = re.I)
    table = re.sub("\s+([A-Z0-9]{1,2})\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" \1-\2-\3 ", table, flags = re.I)
    table = re.sub("\s+([A-Z0-9]{1,2})\.*-\.*([A-Z]{3})\.*-\.(\d{2,4})", r" \1-\2-\3 ", table, flags = re.I)
    table = re.sub("\s+OS\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" 05-\1-\2 ", table, flags = re.I)
    table = re.sub("\s+Ol\s*-\s*([A-Z]{3})\s*-\s*(\d{2,4})", r" 01-\1-\2 ", table, flags = re.I)
    table = re.sub("\-No\>J\-", r"-Nov-", table, flags = re.I)
    table = re.sub("\.+\s*Qct", r"-Oct", table, flags = re.I)
    table = re.sub("12S|l2S|l25", r"125", table)
    table = re.sub("lOS|l0S|l05|1OS|10S", r"105", table)
    table = re.sub(" 19-Nov ", "19-Nov-14", table)
    table = re.sub("(\d\d)\-(.*)?\-(\d\d)", r"\1-\2-\3", table)
    table = re.sub("(0l!!c)|(0i!!c)", "Dec", table)
    table = re.sub("\-0ec.*?\-", "-Dec-", table)
    table = re.sub("\-Oec\-", "-Dec-", table) 
    table = re.sub("(\d\d)\-([A-Z]{3})\s+", r"\1-\2-9999", table, flags = re.I)
    table = re.sub("(\d\d)\-\s*([A-Z]{3})\s*\-\s*(\d\d)", r"\1-\2-\3", table, flags = re.I)
    table = re.sub("\s*l\s*S", "15", table)
    table = re.sub("\s*l\s*O\s*", "10", table)
    table = re.sub("OB", r"08", table, flags = re.I)
    table = re.sub("f|j\s*a\s*n", "Jan", table, flags = re.I)
    table = re.sub("O!Han", "01-Jan", table, flags = re.I)
    table = re.sub("0i!!!c", "Jan", table, flags = re.I)
    table = re.sub("\d\-\d\s*\-[a-z]{3}\-14", "27-Dec-14", table, flags = re.I)
    table = re.sub("\-vi", "-14", table, flags = re.I)
    table = re.sub("(\d\d)(\d\d)+\-(\d\d)\-(\d\d*)", r"\1-Dec-\3", table, flags = re.I)
    table = re.sub("(\d\d)(\d\d)+\-([a-z]){2,}\-(\d\d)", r"\1-\2-\3", table, flags = re.I)
    table = re.sub("(\d\d)(\d\d)+\-(\d\d)(\.|\:|,|;)*\-(\d\d*)", r"\1-Dec-\3", table, flags = re.I)
    table = re.sub("(\d\d)\-([a-z]{2,})(\-\d{2,})+", r"\1-Dec-14", table, flags = re.I)
    table = re.sub("\-\s*0\s*e\s*c\s*\-", "-Dec-", table, flags = re.I)
    table = re.sub("Dcc", "Dec", table, flags = re.I)
    table = re.sub("([a-z]{3})\-1", r"\1-14", table, flags = re.I)
    table = re.sub("([a-z]{3})\-144", r"\1-14", table, flags = re.I)
    table = re.sub("De", "Dec", table, flags = re.I)
    table = re.sub("Decc", "Dec", table, flags = re.I)
    table = re.sub("(\d+)\-Dec$", r"\1-Dec-14", table, flags = re.I)
    table = re.sub("\:\-", "-", table, flags = re.I)
    table = re.sub("Janan", "Jan", table, flags = re.I)
    table = re.sub("1\s*\.*S|l\.*\s*S|\]\s*\.*S", "15", table, flags = re.I)
    table = re.sub("2S", "25", table, flags = re.I)    
    table = re.sub("os\.|oa\.", "05-", table, flags = re.I)
    table = re.sub("J\.i\:n|J\s*\.*;\s*\.*m|J\s*\.*l\s*\.*n|j\s*\.*t\s*\.*n|j\s*\.*t\s*\.*m|j;*:*m|j:*;*m|jo1n|j01n|join|joln|j0ln", "Jan", table, flags = re.I)
    table = re.sub("l\s*\.*l", "11", table, flags = re.I)
    table = re.sub("l(\d)", r"1\1", table, flags = re.I)
    table = re.sub("(\d)l", r"\1(1)", table, flags = re.I)
    table = re.sub(">|<|\s*..._\s*|\-n", "", table, flags = re.I)
    table = re.sub("([a-z]{3})(\d\d)", r"\1-\2", table, flags = re.I)
    table = re.sub("\-(\d{3,})", "14", table)
    ## more rules
    table = re.sub("\-([a-z]{3})(\d{2})", r" \1-\2", table, flags = re.I)
    table = re.sub("\s(\d+)\-Au", r" \1-Aug-14", table, flags = re.I)
    table = re.sub("\s(\d+)\-Ju", r" \1-Jun-14", table, flags = re.I)
    table = re.sub("\s(\d+)\-Ap", r" \1-Apr-14", table, flags = re.I)
    table = re.sub("\s(\d+)\s+([A-Z]{3})\-(\d\d)", r" \1-\2-\3", table, flags = re.I)
    table = re.sub("\sB\s*\-*([A-Z]{3})\-(\d\d)", r" 08-\1-\2", table, flags = re.I)
    table = re.sub("\sg\s*\-*\d\d", "", table, flags = re.I)
    table = re.sub("\dg\s*\-*\s*(\d\d)", r"01-Aug-\1", table, flags = re.I)
    table = re.sub("\sS\s*\-*([A-Z]{3})\-(\d\d)", r" 05-\1-\2", table, flags = re.I)
    table = re.sub("\s(\d)\s*\-*\s*([A-Z]\s*[A-Z]\s*[A-Z])\s*\-*\s*(\d\d)", r" \1-\2-\3", table, flags = re.I)
    table = re.sub("\-*\s*([A-Z]\s*[A-Z]\s*[A-Z])\s*\-*\s*", r"-\1-", table, flags = re.I)
    table = re.sub("(\d)\s*\-\s*A ug\-(\d\d)", r"\1-Aug-\2", table, flags = re.I)
    table = re.sub("(\d)\s*\-\s*([A-Z]{3})\-(\d\d)\\\-\d\d", r"\1-\2-\3", table, flags = re.I)
    table = re.sub("\-[a-z]+(\d\d)", r"-\1", table, flags = re.I)
    table = re.sub("\-\s*J\s*U\s*L\s*\-|\-\s*J\s*\-|\-\s*U\s*L\s*\-", "-Jul-", table, flags = re.I)
#     table = re.sub("\-\s*M\s*A\s*Y\s*\-|\-\s*M\s*A\s*\-|\-\s*A\s*Y\*\-|\-\s*M\s*\-", "-May-", table, flags = re.I)
    table = re.sub("\s+a\s*y\s*-\s*(\d+)\s*", r"01-May-\1", table, flags = re.I)
    table = re.sub("\s+l\s*-\s*(\d+)\s*", r"01-Jul-\1", table, flags = re.I)
    table = re.sub("(\d)r\-\d+", r"\1", table)
    
    
    '''
    ## Rows ##
    '''
    for row in [x for x in table[2:].splitlines() if x]:
      row = row.strip()
      row = re.sub("-\\s*$", "", row)
      row = re.sub("^.*?\\s+", "", row)
      row = re.sub("^\\s*\\w+\\s+", "", row)
      row = re.sub("\s+[0-9-_]+$", "", row)
      row = re.sub("\s+", " ", row)
      row = re.sub("(\d+)\-([a-z]+)$\-*", r"\1-\2-14", row, flags = re.I)
      row = re.sub("\-J\-", "Jun", row, flags = re.I)
      row = re.sub("\-\s*Ma\s*\-|\-\s*M\s*\-", "May", row, flags = re.I)
      row = re.sub("(\d+)([A-Z]{3})(\d{2,})", r"\1-\2-\3", row, flags = re.I)
      row = re.sub("M01-May-1", r"May-14", row, flags = re.I)
      
      row = re.sub("\s+\-\s*([A-Z]{3})\s*\-\s*(\d{2})", r"01-\1-\2", row, flags = re.I)
      row = re.sub("\d\(\d\)", "01", row, flags = re.I)
      row = re.sub("\-([A-Z]{3})\s*\-\s*[A-Z]+", r"\1-14", row, flags = re.I)
      row = re.sub("\-\s*(\d+)l\s*\-\s*\d+", r"-\1", row, flags = re.I)
      row = re.sub("(\d+)o", r"\1", row, flags = re.I)
      row = re.sub("!0", "10", row)
      row = re.sub("0ct", "Oct", row, flags = re.I)
      row = re.sub("(\d)ov", r"\1-Nov-", row, flags = re.I)
      row = re.sub("0ct", "Oct", row, flags = re.I)
      row = re.sub("(\d+)S", r"\1", row, flags = re.I)
      
      # Very specific rules for one file
      row = re.sub("11-1ul-16-Dec-14", "11-Jul-14 16-Dec-14", row, flags = re.I)
      row = re.sub("15-117-Jun-14", "15-Jul-14 17-Jun-14", row, flags = re.I)
      row = re.sub("227-Sov-14v14", "22-Nov-14 01-Nov-14", row, flags = re.I)
      row = re.sub("(\d+)\s*v\s*\-*\s*(\d+)", r"\1-Nov-\2", row, flags = re.I)
      row = re.sub("l\-Dec\-14\-Dec\-15", "01-Dec-14 01-Dec-15", row, flags = re.I)
      row = re.sub("11\-1ul\-16\-Dec\-14", "11-Jul-16 01-Dec-14", row, flags = re.I)
      row = re.sub("(\d+)V\-15\-Dec\-14", r"\1-Nov-15 01-Dec-14", row, flags = re.I)      
      row = re.sub("(\d+)\-([A-Z]+)\-(\d+)\s+\d+", r"\1-\2-\3", row, flags = re.I)
      row = re.sub("\\\-16", "", row, flags = re.I)
      row = re.sub("([A-Z]+)\s*\-\s*(\d+)\s+\d+", r"\1-\2", row, flags = re.I)
      row = re.sub("(\d)(\d{2})\-Sov\-14\-Nov", r"\1-\2-Nov-14 01-Nov", row, flags = re.I)
      row = re.sub("12\-Jun\-14I\-16 l 4l", "12-Jun-14 01-Jul-16", row, flags = re.I)
      row = re.sub("l-Dec-15-Dec-15", "01-Dec-15 01-Dec-15", row, flags = re.I)
       

      # Fix space in month names
      row = re.sub("\s*J\s*\-*\s*A\s*\-*\s*N", "Jan", row, flags = re.I)
      row = re.sub("\s*F\s*\-*\s*E\s*\-*\s*B", "Feb", row, flags = re.I)
      row = re.sub("\s*M\s*\-*\s*a\s*\-*\s*r", "Mar", row, flags = re.I)
      row = re.sub("\s*A\s*\-*\s*P\s*\-*\s*R", "Apr", row, flags = re.I)
      row = re.sub("\s*M\s*\-*\s*A\s*\-*\s*Y", "May", row, flags = re.I)
      row = re.sub("\s*J\s*\-*\s*U\s*\-*\s*N", "Jun", row, flags = re.I)
      row = re.sub("\s*J\s*\-*\s*U\s*\-*\s*L", "Jul", row, flags = re.I)
      row = re.sub("\s*A\s*\-*\s*U\s*\-*\s*G", "Aug", row, flags = re.I)
      row = re.sub("\s*S\s*\-*\s*E\s*\-*\s*P", "Sep", row, flags = re.I)
      row = re.sub("\s*O\s*\-*\s*C\s*\-*\s*T", "Oct", row, flags = re.I)
      row = re.sub("\s*N\s*\-*\s*O\s*\-*\s*V", "Nov", row, flags = re.I)
      row = re.sub("\s*D\s*\-*\s*E\s*\-*\s*C", "Dec", row, flags = re.I)
      
      row = re.sub("\s\-Ju", "", row, flags = re.I)
      
      
#       -Jul-16
# 3(1)-Jul-14
# 12-Jan-e
# 12-Jun-14l- 16
# 3-M ar-16
#  15-117-Jun-14
#  !0-Aug-14
#  19o
#  20o
#  1-B Ja-n-14
#  2-0ct-15
#  27ov-l
#  27ov-14S
#  22OV-14S
#  7ov14
#  223-Sov-14 7ov-14 S
    
      row = row.strip()
            
      if len(row) > 55  and\
        hasNumbers(row):
        
        print("--> Raw row:%s"%row)
        
#         Initially Last column is:8-Au
# Both date columns missing
# splitting again and the last column is 99-Jan-9999 and second last column is 8-Au
# second last column: 8-Au
  
        # Check if the second last columna is a date columns or not
        # In some cases the last date column is not read
        row_cols = row.split(' ')
        
        last_col = row_cols[len(row_cols)-1].strip()
        second_last_col = row_cols[len(row_cols)-2].strip()
        
        print("Initially Last column is:{}".format(last_col))
        # if last column is not a date - add last and second last date columns
        if (not bool(re.match("\d{1,2}\-[a-z]{3}\-\d\d", last_col, flags = re.I))):
          print("Both date columns missing")
          # append the available date column at the end of the row
          row = row + " 99-Jan-9999"
        
        row_cols = row.split(' ')
        
        last_col = row_cols[len(row_cols)-1].strip()
        second_last_col = row_cols[len(row_cols)-2].strip()
        print("splitting again and the last column is {0} and second last column is {1}".format(last_col,second_last_col))
        if (bool(re.match("\s*\d{1,2}\-[a-z]{3}\-\d{2,4}\s*", second_last_col, flags = re.I))):
          print("second last column is a date")
                     
                     
        # Second last column is not a date but last column is
        if (not bool(re.match("\s*\d{1,2}\-[a-z]{3}\-\d{2,4}\s*", second_last_col, flags = re.I)) and\
                    bool(re.match("\s*\d{1,2}\-[a-z]{3}\-\d{2,4}\s*", last_col, flags = re.I))):
          print("second last column: %s"%second_last_col)
          # append the available date column at the end of the row
          row = row + " "\
            + str(int(last_col.split("-")[0]) + 1) + "-"\
            + last_col.split("-")[1] + "-"\
            + last_col.split("-")[2]
          
      

        print(row)
        print("----------------------------------")

        global r
        r = row
        col_num = 0

        for col_name in col_names:
          col_val = get_col()
          cols[col_name] = col_val
          col_val = re.sub(r"([0-9]+)_", "\g<1>",col_val)
          col_val = re.sub(r"_([0-9]+)", "\g<1>",col_val)
          col_val = re.sub(r"\s+-([0-9]+)", " \g<1>",col_val)
          col_val = re.sub(r"([0-9]+)-\s+", "\g<1> ",col_val)

          # Write in spreadsheet
          cell = cells[col_num] + str(sheet_row_num)    
          sheet[cell].value= col_val
#           print("Col Name:{0}, col value:{1} written in cell {2}".format(col_name, col_val,cell))
          col_num = col_num + 1

        r = re.sub("\\s+", "_", r)
        cols["spool_no"] = r
        cell ="D"+str(sheet_row_num)    
        sheet[cell].value = r

        cell ="C"+str(sheet_row_num)    
        sheet[cell].value = page_num

        sheet_row_num = sheet_row_num + 1
  except Exception:
    print("Exception on page%i" %page_num)
    traceback.print_exc()
    pass
  
#   print("page_num:{0}, total_pages:{1}".format(page_num, total_pages))

  if(page_num == total_pages):
    break
          
  page_num = page_num + 1
  page_num_printed = page_num_printed + 1

# print("cell d2 value {}".format(sheet["d2"].value))
# wb.save("/content/gdrive/My Drive/Colab Notebooks/data/excel_instructions.xlsx")    
# wb.save("/content/gdrive/My Drive/Colab Notebooks/data/20files/L290-AQ-MDR-13850_0_SECTN_89OF107_IPX Reviewed_AC-OCR.xlsx")
# wb.save("/content/gdrive/My Drive/Colab Notebooks/data/test.xlsx")



Page#1
Page num printed#10795
--> Raw row:PG014-01A3-WMP-14 109 116 130 135 121 230 255 236 248 263 246 6-Aug-14-Aug-14
Initially Last column is:6-Aug-14-Aug-14
splitting again and the last column is 6-Aug-14-Aug-14 and second last column is 246
second last column: 246
PG014-01A3-WMP-14 109 116 130 135 121 230 255 236 248 263 246 6-Aug-14-Aug-14 7-Aug-14
----------------------------------
Page#2
Page num printed#10796
--> Raw row:25 255 264 269 270 262 278 288 290 299 300 291 25-Dec-14-Dec-14
Initially Last column is:25-Dec-14-Dec-14
splitting again and the last column is 25-Dec-14-Dec-14 and second last column is 291
second last column: 291
25 255 264 269 270 262 278 288 290 299 300 291 25-Dec-14-Dec-14 26-Dec-14
----------------------------------
--> Raw row:PG004-6CA3WMP-14 266 278 290 280 266 276 288 299 304 288 299 296 25-Dec-14-Dec-14
Initially Last column is:25-Dec-14-Dec-14
splitting again and the last column is 25-Dec-14-Dec-14 and second last column is 296
second last column:

Traceback (most recent call last):
  File "<ipython-input-30-0ec5c41e5fdf>", line 58, in <module>
    end_of_table = end_of_table.start() - skip
AttributeError: 'NoneType' object has no attribute 'start'


Initially Last column is:2-Aug-14-Aug-14
splitting again and the last column is 2-Aug-14-Aug-14 and second last column is 244
second last column: 244
A1EN-V_003-3CB3-02-WMP-14 140 119 146 119 127 211 270 243 248 248 244 2-Aug-14-Aug-14 3-Aug-14
----------------------------------
--> Raw row:A1EN-V 003 -3CB3-03-WMP-14 134 103 142 120 122 222 284 278 236 235 251 2-Aug-14-Aug-14
Initially Last column is:2-Aug-14-Aug-14
splitting again and the last column is 2-Aug-14-Aug-14 and second last column is 251
second last column: 251
A1EN-V 003 -3CB3-03-WMP-14 134 103 142 120 122 222 284 278 236 235 251 2-Aug-14-Aug-14 3-Aug-14
----------------------------------
--> Raw row:V 003-3CB3-05-WMP-14 135 114 111 134 124 220 290 230 208 256 241 2-Aug-14-Aug-14
Initially Last column is:2-Aug-14-Aug-14
splitting again and the last column is 2-Aug-14-Aug-14 and second last column is 241
second last column: 241
V 003-3CB3-05-WMP-14 135 114 111 134 124 220 290 230 208 256 241 2-Aug-14-Aug-14 3-Aug-14
-------

Traceback (most recent call last):
  File "<ipython-input-30-0ec5c41e5fdf>", line 58, in <module>
    end_of_table = end_of_table.start() - skip
AttributeError: 'NoneType' object has no attribute 'start'



A1EP14-Jan-14-1GA1-01-WMP-4 115 115 115 110 115 177 178 189 178 184 181 11-Dec-14-Dec-14 12-Dec-14
----------------------------------
--> Raw row:A1EP14-Jan-14-1GA1-01-WMP-5 107 109 106 110 108 186 187 179 186 167 181 11-Dec-14-Dec-14
Initially Last column is:11-Dec-14-Dec-14
splitting again and the last column is 11-Dec-14-Dec-14 and second last column is 181
second last column: 181
A1EP14-Jan-14-1GA1-01-WMP-5 107 109 106 110 108 186 187 179 186 167 181 11-Dec-14-Dec-14 12-Dec-14
----------------------------------
--> Raw row:A1EP14-Jan-14-1GA1-01-WMP-6 107 105 124 105 109 167 189 184 168 189 179 11-Dec-14-Dec-14
Initially Last column is:11-Dec-14-Dec-14
splitting again and the last column is 11-Dec-14-Dec-14 and second last column is 179
second last column: 179
A1EP14-Jan-14-1GA1-01-WMP-6 107 105 124 105 109 167 189 184 168 189 179 11-Dec-14-Dec-14 12-Dec-14
----------------------------------
--> Raw row:A1EP14-Jan-14-1GA1-01-WMP-7 120 105 119 123 116 198 184 185 150 178 179 11-Dec-

Traceback (most recent call last):
  File "<ipython-input-30-0ec5c41e5fdf>", line 58, in <module>
    end_of_table = end_of_table.start() - skip
AttributeError: 'NoneType' object has no attribute 'start'


Page#386
Page num printed#11180
--> Raw row:r--t---------------t--t--'-t-2-r-l~r--E~-'-i-2-i-'-t--t-'-t-A-VE-t-' -t-2-r-l~r--~E~-1 A1ER14 OG004-01A3-WMP-14 138 141 125 143 13 265 241 239 255 281 256 29-Dec-14-Dec-14
Initially Last column is:29-Dec-14-Dec-14
splitting again and the last column is 29-Dec-14-Dec-14 and second last column is 256
second last column: 256
r--t---------------t--t--'-t-2-r-l~r--E~-'-i-2-i-'-t--t-'-t-A-VE-t-' -t-2-r-l~r--~E~-1 A1ER14 OG004-01A3-WMP-14 138 141 125 143 13 265 241 239 255 281 256 29-Dec-14-Dec-14 30-Dec-14
----------------------------------
Page#387
Page num printed#11181
--> Raw row:OG005-1CA1-WMP-14 136 132 128 129 134 257 243 276 242 289 261 23-Dec-14
Initially Last column is:23-Dec-14
splitting again and the last column is 23-Dec-14 and second last column is 261
second last column: 261
OG005-1CA1-WMP-14 136 132 128 129 134 257 243 276 242 289 261 23-Dec-14 24-Dec-14
----------------------------------
Page#388
Page num printed#11182
--> Raw row: